In [ ]:
# import required libraries
import cv2 as cv
import numpy as np
import os
import matplotlib.pyplot as plt

def detect_face(img):
    '''
    function to detect face
    '''
    
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    face_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5)

    # if no faces are detected then return original img
    if len(faces) == 0:
        return None, None

    # under the assumption that there will be only one face,
    # extract the face area
    (x, y, w, h) = faces[0]

    # return the face part of the image, coordinate points to draw rectangle
    return gray[y:y + w, x:x + h], faces[0]

def prepare_training_data(data_folder_path):
    '''function for data preparing'''
#     get the dictionary in data folder
    dirs = os.listdir(data_folder_path)
    
#     list to hold all subject faces
    faces = []
    
#     list of labels of all subjects
    lables = []

#     let's go through each directory and get all imgs within it
    for dir_names in dirs:
        
#         because our dir names start with "a" so
        if not dir_names.startswith("a"):
            continue
        
#         remove letter 'a' to give label
        lable = int(dir_names.replace('a', ''))
    
#         get subject data part
        subject_data_path = data_folder_path + '/' + dir_names
        
#         get the images name inside the subject directory
        subject_image_name = os.listdir(subject_data_path)
        
#         ignore cache file
        for image_names in subject_image_name:
            if image_names.startswith('.'):
                continue
                
#             build image path
            image_path = subject_data_path + '/' + image_names
            
#             read image
            image = cv.imread(image_path)
            
#             show image
            cv.imshow('Training Images...', cv.resize(image, (400,500)))
            
            cv.waitKey(100)
#             detect face
            face, rect = detect_face(image)
    
#             ignore if the face is not found
            if face is not None:
                
#                 add face to the list of faces
                faces.append(face)
    
#                 add lables for this face
                lables.append(lable)
    cv.waitKey(1)
    cv.destroyAllWindows()
    
    return faces, lables

print('Preparing data...')
faces, labels = prepare_training_data('training-data/')
print('Data prepared')

print('total faces', len(faces))
print('tatal lables', len(labels))

# subjects
subjects = [" ", "Myo Min Aung", "Steve Job"] #0 for blank

# create LBPH face recognizer
face_recognizer = cv.face.LBPHFaceRecognizer_create()

# train our face recognizer of our training face
face_recognizer.train(faces, np.array(labels))

def draw_rectangle(img, rect):
    """function to draw rectangle"""
    (x,y,w,h) = rect
    cv.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 4)

def write_text(img, text, x, y):
    '''function to write text'''
    cv.putText(img, text, (x,y), cv.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)

def predict(test_img):
    '''function to recognize the person in input image and draw a rectangle on his face'''

    # make a copy of original image because we don't want to do on original image
    img = test_img.copy()

    # detect face from the image
    face, rect = detect_face(img)

    # predict image using our face recognizer
    label = face_recognizer.predict(face) #it returns only labels

    # get the names of respective labels returned by the face recognizer
    label_text = subjects[label[0]]
    
    # draw rectangle around the detected face 
    draw_rectangle(img, rect)

    # draw name of predict person
    write_text(img, label_text, rect[0], rect[1] - 5)

    # return result image
    return img

# load test images
test_img1 = cv.imread("test-data/test1.jpg")
test_img2 = cv.imread("test-data/test2.jpg")

# perform a predection
predicted_img1 = predict(test_img1) 
predicted_img2 = predict(test_img2)
print('predection completes')

# show image
cv.imshow("1", predicted_img1)
cv.imshow("2", predicted_img2)

images = [predicted_img1, predicted_img2]
lables = [subjects[1], subjects[2]]

# show image with matplotlib 
for i in range(len(images)):
    plt.subplot(2, 2, i + 1), plt.imshow(images[i], 'gray')
    plt.title(labels[i])
    plt.xticks([]), plt.yticks([])
    plt.show()

cv.waitKey(0) & 0xff == ord('q')
cv.destroyAllWindows()

In [ ]:
# Opens the Video file
import cv2

cap= cv2.VideoCapture('person1_video/myominaung.mp4')
i=0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    cv2.imwrite(str(i)+'.jpg',frame)
    i+=1
    
    if cv2.waitKey(1) & 0xff == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()